## DL3 skymap with gammapy
author:
* Tarek Hassan (thassan@ifae.es)
* Cosimo Nigro (cosimonigro2@gmail.com)

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from os import listdir
from os.path import isfile, join

import astropy.units as u
from astropy.coordinates import SkyCoord
from astropy.convolution import Ring2DKernel, Tophat2DKernel
from astropy.visualization import simple_norm

from gammapy.data import DataStore
from gammapy.image import SkyImage, SkyImageList
from gammapy.detect import KernelBackgroundEstimator as KBE

In [ ]:
import gammapy

In [ ]:
gammapy.__version__

In [ ]:
dl3Dir = "../VEGAS/DL3med/"
dl3Files = [dl3Dir + f for f in listdir(dl3Dir) if isfile(join(dl3Dir, f)) and f.endswith("fits")]
print(dl3Files)

# Use pyV3DL3's helper functions to get DataStore
## 1. Generate index file and generate from directory

In [ ]:
from pyV2DL3.generateObsHduIndex import create_obs_hdu_index_file

In [ ]:
# Create index files
create_obs_hdu_index_file(dl3Files,index_file_dir='./')
data_store = DataStore.from_dir('./')

## 2. Generate DataStroe using getDSfromList
The index tables are generated in memory and not saved to files.

In [ ]:
from pyV2DL3.gammapy_util import getDSfromList

In [ ]:
source_pos = SkyCoord(83.633083, 22.0145, unit='deg')
# If you have internet access, you could also use this to define the `source_pos`:
# source_pos = SkyCoord.from_name('crab')
print(source_pos)

In [ ]:
ref_image = SkyImage.empty(
    nxpix=400, nypix=400, binsz=0.02,
    xref=source_pos.ra.deg, yref=source_pos.dec.deg,
    coordsys='CEL', proj='TAN',
)


In [ ]:
# Make a counts image for a single observation
events = data_store.obs(obs_id=54809).events
counts_image = SkyImage.empty_like(ref_image)
counts_image.fill_events(events)

In [ ]:
norm = simple_norm(counts_image.data, stretch='sqrt', min_cut=0, max_cut=0.7)
counts_image.smooth(radius=0.1 * u.deg).plot(norm=norm, add_cbar=True)

In [ ]:
# we can also take a look at how much is the value of the map in one position, say the nominal position of the Crab
from astropy.coordinates import SkyCoord
counts_image.lookup(source_pos)

In [ ]:
# Making a counts image for multiple observations is a bit inconvenient at the moment
counts_image2 = SkyImage.empty_like(ref_image)
for obs_id in data_store.obs_table['OBS_ID'].data:
    events = data_store.obs(obs_id=obs_id).events
    counts_image2.fill_events(events)

In [ ]:
norm = simple_norm(counts_image2.data, stretch='sqrt', min_cut=0, max_cut=0.5)
counts_image2.smooth(radius=0.1 * u.deg).plot(norm=norm, add_cbar=True)

In [ ]:
# suppose now we want to take a look at a zoomed part of the countmap 
# 1.5 degree around the source nominal postition
size = u.Quantity([1.5, 1.5], 'deg')
cutout = counts_image2.cutout(source_pos, size)
cutout.show()

## Background modeling
In Gammapy a few different methods to estimate the background are available.

Here we'll use the gammapy.detect.KernelBackgroundEstimator to make a background image and the make a significance image.

Starting from an initial background estimate and exclusion mask (both provided, optionally) the algorithm works as follows:

* Compute significance image
* Create exclusion mask by thresholding significance image
* Compute improved background estimate based on new exclusion mask

The steps are executed repeatedly until the exclusion mask does not change anymore.



In [ ]:
source_kernel = Tophat2DKernel(radius=5)
source_kernel.normalize(mode='peak')
source_kernel = source_kernel.array

background_kernel = Ring2DKernel(radius_in=20, width=20)
background_kernel.normalize(mode='peak')
background_kernel = background_kernel.array

In [ ]:
plt.imshow(source_kernel, interpolation='nearest', cmap='gray')
plt.colorbar()
plt.grid('off')

In [ ]:
plt.imshow(background_kernel, interpolation='nearest', cmap='gray')
plt.colorbar()
plt.grid('off')

In [ ]:
# To use the `KernelBackgroundEstimator` you first have to set
# up a source and background kernel and put the counts image input
# into a container `SkyImageList` class.
images = SkyImageList()
images['counts'] = counts_image2

kbe = KBE(
    kernel_src=source_kernel,
    kernel_bkg=background_kernel,
    significance_threshold=5,
    mask_dilation_radius=0.06 * u.deg,
)
print(images)
# This takes about 10 seconds on my machine
result = kbe.run(images)

In [ ]:
background_image = result['background']
norm = simple_norm(background_image.data, stretch='sqrt')#, min_cut=0, max_cut=0.5)
background_image.plot(norm=norm, add_cbar=True)

In [ ]:
result['exclusion'].plot()

In [ ]:
significance_image = result['significance']
significance_image.plot(add_cbar=True)
# significance_image.plot(add_cbar=True, vmin=-5, vmax=5)

In [ ]:
np.nanmax(significance_image.data[np.isfinite(significance_image.data)])